In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# HP Model
B=list('AILMFPWV')
U=list('RHK')
X=list('DE')
Z=list('NCQGSTY')

# HC Model
# B=list('RNDQEKH')
# U=list('LIVAMF')
# X=list('STYW')
# Z=list('CGP')

In [ ]:
def cnvrt(S):
    for i in range(len(S)):
        for j in range(len(B)):
            if S[i] == B[j]:
                S[i]='B'
        for j in range(len(U)):
            if S[i] == U[j]:
                S[i]='U'
        for j in range(len(X)):
            if S[i] == X[j]:
                S[i]='X'
        for j in range(len(Z)):
            if S[i] == Z[j]:
                S[i]='Z'  
    return S

In [ ]:
data = pd.read_csv("36_primary.csv")
df=pd.DataFrame(data)
Sequence = df['Sequence'].values.tolist()
# print (df['Sequence'].iloc[0])

In [ ]:
df1=df

In [ ]:
for i in range(len(df)):
    df1['Sequence'].iloc[i]=''.join(cnvrt(list(df['Sequence'].iloc[i])))

In [ ]:

B = [-1,0]
U = [0,1]
X = [1,0]
Z = [0,-1]
xi =[]
yi = []
mi = []
mixi =[]
miyi = []

n=len(Sequence)

i=0
for ch in Sequence:
#Calculation of xi,yi
  if ch=='B':
    if(i==0):
      xi.insert(i, B[0])
      yi.insert(i, B[1])
    else:
      xi.insert(i, B[0]+xi[i-1])
      yi.insert(i, B[1]+yi[i-1])
        
  elif ch=='U':
    if(i==0):
      xi.insert(i, U[0])
      yi.insert(i, U[1])
    else:
      xi.insert(i, U[0]+xi[i-1])
      yi.insert(i, U[1]+yi[i-1])
        
  elif (ch=='X'):
    if(i==0):
      xi.insert(i, X[0])
      yi.insert(i, X[1])
    else:
      xi.insert(i, X[0]+xi[i-1])
      yi.insert(i, X[1]+yi[i-1])
  
  elif (ch=='Z'):
    if(i==0):
      xi.insert(i, Z[0])
      yi.insert(i, Z[1])
    else:
      xi.insert(i, Z[0]+xi[i-1])
      yi.insert(i, Z[1]+yi[i-1])
        
  #calculation of mi
  mi.insert(i, 1)
  if i>0:
      for j in range(i-1, -1, -1):
          if xi[i]==xi[j] and yi[i]==yi[j]:
              mi[i] = mi[j]+1
              break
          
  #calculation of mixi
  mixi.insert(i, mi[i]*xi[i])
    
  #calculation of miyi
  miyi.insert(i, mi[i]*yi[i])
    
  print('[','(',xi[i],',',yi[i],')',mi[i],']')
  
  i=i+1

In [ ]:
#calculation of ux, uy, uz
ux = sum(mixi)/sum(mi)
uy = sum(miyi)/sum(mi)
print('\n','ux:',ux,'\n', 'uy:',uy)

In [ ]:
ixx=iyy=ixy=0

for i in range(n):
  ixx=ixx+(mi[i]*(xi[i]**2))

for i in range(n):
  iyy=iyy+(mi[i]*(yi[i]**2))

for i in range(n):
  ixy=ixy+(mi[i]*xi[i]*yi[i])
  ixy = ixy * (-1)

matrix = np.array([[ixx, ixy],
                   [ixy, iyy]])
print('*Matrix:*')
print('\n',matrix,'\n')
# Calculate eigenvalues
I1, I2 = np.linalg.eigvals(matrix)
print('*Eigenvalues:*')
print('\n','I1:', I1, '\n', 'I2:',I2)

In [ ]:
#final vector calculation
v = [ux/I1, uy/I1,  ux/I2, uy/I2]
# print('*Vectors:*', '\n', v)

In [ ]:
# Calculate pairwise Manhattan distance between rows
def pairwise_manhattan_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = np.sum(np.abs(df.iloc[i] - df.iloc[j]))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

# Calculate pairwise Euclidean distance between rows
def pairwise_euclidean_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = np.sqrt(np.sum((df.iloc[i]-df.iloc[j])**2))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

# Calculate pairwise standard distance between rows
def pairwise_standard_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = np.sum(np.abs(df.iloc[i]-df.iloc[j]))/(1+(np.sum(np.abs(df.iloc[i]-df.iloc[j]))))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

def pairwise_maximum_distance(df):
    n = df.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i, j] = max(np.abs(df.iloc[i]-df.iloc[j]))
            # distance_matrix[j, i] = distance_matrix[i, j]  # Distance matrix is symmetric
    return distance_matrix

In [ ]:
dmat_mh = pd.DataFrame(pairwise_manhattan_distance(pd.DataFrame(v)))
dmat_ed = pd.DataFrame(pairwise_euclidean_distance(pd.DataFrame(v)))
dmat_sd = pd.DataFrame(pairwise_standard_distance(pd.DataFrame(v)))
dmat_md = pd.DataFrame(pairwise_maximum_distance(pd.DataFrame(v)))

In [ ]:
dmat=pd.concat([dmat_mh,dmat_ed,dmat_sd,dmat_md])
dmat_ed.to_csv('result.csv')